# Usage

In [ ]:
from importlib import reload  ## only for developping
import os
from pathlib import Path
import numpy as np

from obspy.clients.filesystem.sds import Client
from obspy.core import UTCDateTime as UTC

from data_quality_control import sds_db, dqclogging, util

In [ ]:
#logger = sds_db.logger
dqclogging.configure_handlers(sds_db.logger, "INFO", "DEBUG", "usage_demo.log")

## Define parameters

In [ ]:
# NSLC
nslc_code = "GR.BFO..BHZ"

# Processing parameters
overlap = 60 
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

# Data sources
#sds_root = os.path.abspath('../sample_sds/')
sds_root = Path('../sample_sds/')
inventory_routing_type = "eida-routing"

# Output configuration
outdir = Path('../sample_output/show_processing/')
fileunit = "year" # period to store in 1 file

You can use the sds-Client directly to check the content
of the database. Note though, that this can take some time
if your database is large.

In [ ]:
sdsclient = Client(sds_root)
sdsclient.get_all_nslc()

## Process raw data

Since we have an sds-database, we use the `sds_db` module to extract amplitudes and
power spectral densities (PSD) from the raw seismic data.

In [ ]:
reload(sds_db)  # only for developping
processor = sds_db.SDSProcessor(
        nslc_code=nslc_code,
        sds_root=sds_root,
        inventory_routing_type=inventory_routing_type,
        outdir=outdir, 
        fileunit=fileunit,
        # Processing parameters
        overlap=overlap, nperseg=nperseg, 
        winlen_seconds=winlen_in_s, 
        proclen_seconds=proclen,
        amplitude_frequencies=(fmin, fmax))

print(processor)
#processor.logger.setLevel("INFO")

In [ ]:
startdate = UTC("2020-12-20")
enddate = UTC("2021-01-15")

In [ ]:
%%time
#it -n1 -r7
processor.process(startdate, enddate, force_new_file=True)

If we change the `fileunit` to `"month"`, we get different filenames, indicating
also the month.

In [ ]:
reload(sds_db)  # only for developping
processor = sds_db.SDSProcessor(
        nslc_code,
        sds_root=sds_root,
        inventory_routing_type=inventory_routing_type,
        outdir=outdir, 
        fileunit="month",
        # Processing parameters
        overlap=overlap, nperseg=nperseg, 
        winlen_seconds=winlen_in_s, 
        proclen_seconds=proclen,
        amplitude_frequencies=(fmin, fmax))

print(processor)

In [ ]:
startdate = UTC("2020-12-20")
enddate = UTC("2021-01-15")

In [ ]:
%%time
#it -n1 -r7
processor.process(startdate, enddate, force_new_file=True)

`filunit="month"` produces output files with ending `YYYY-MM.hdf5`. Note that 
these files are only about 1/12 of the size of the yearly files, indicating
that they cover only one month rather than 1 year of data.

In [ ]:
%ls -lh ../sample_output/show_processing/

In [ ]:
f = Path("../sample_output/show_processing/GR.BFO..BHZ_2020-12.hdf5")

In [ ]:
f.stat().st_size

## Analyze processed data

With the `analysis` module, the processed data can be accessed 
and visualized.

In [ ]:
from data_quality_control import analysis #, util

In [ ]:
#logger = sds_db.logger
#dqclogging.configure_handlers(analysis.logger, "INFO", "DEBUG", "usage_demo.log")

In [ ]:
# Only for display in documentation!
from IPython.core.display import display, HTML 

In [ ]:
reload(analysis)
#reload(util)
lyza = analysis.Analyzer(outdir, nslc_code,
                            fileunit="year")

In [ ]:
print(lyza)

In [ ]:
files = lyza.get_available_datafiles()
print(files)

In [ ]:
lyza.get_available_timerange()

## View data for time range

In [ ]:
startdate = UTC("2020-12-25")
enddate = UTC("2021-01-15")

In [ ]:
DATA = lyza.get_data(startdate, enddate)

In [ ]:
print(DATA)

In [ ]:
print(lyza)

#### Spectrogram

In [ ]:
fig = lyza.plot_spectrogram()

#### 3D-Plots

In [ ]:
fig_amp, fig_psd = lyza.plot3d()

In [ ]:
display(HTML(fig_psd.to_html(include_mathjax="cdn")))

In [ ]:
display(HTML(fig_amp.to_html(include_mathjax="cdn")))

In a notebook or script you could simply run:

```python
fig.show()
```

### View data for selected times

For some use cases one might want to get only the power 
spectral densities for specific times. For example, one could
filter a time series of wind speed data for times with a
certain speed. 

The Analyzer extracts psds for specific times only if it
receives a list of UTCDateTimes. 

#### Create random time list

For demonstration, we create a list of 100 random times within
the time range of the data.

In [ ]:
starttime = UTC("2020-12-25")
endtime = UTC("2021-01-10")
times = np.arange(str(starttime.date), str(endtime.date),
             dtype="datetime64[h]")

In [ ]:
tlist = np.sort(np.random.choice(times, 100, replace=False))

tlist = [UTC(str(t)) for t in tlist]

#### Read data for times in list

Since we used 100 datetimes, we get a PSD-Matrix which has
100 entries along the time axis. The amplitude matrix remains
unaffected.

In [ ]:
DATA = lyza.get_data(tlist)

print(lyza.starttime, lyza.endtime)
print(lyza.amps.shape)
print("PSD shape:", lyza.psds.shape)
print("len(tlist):", len(tlist))

In [ ]:
fig = lyza.plot_spectrogram()

#### 3D-Plots

In [ ]:
lyza.psds.shape

In [ ]:
fig_amp, fig_psd = lyza.plot3d()

In [ ]:
display(HTML(fig_psd.to_html(include_mathjax="cdn")))

In [ ]:
display(HTML(fig_amp.to_html(include_mathjax="cdn")))